In [ ]:
import numpy as np
from PIL import Image, ImageSequence
import matplotlib.pyplot as plt
import torch

In [ ]:
def project_theta(theta, m_values):
    projections = []
    for m in m_values:
        sin_m_theta = torch.sin(m * theta)
        cos_m_theta = torch.cos(m * theta)
        projected_vectors = torch.stack((cos_m_theta, sin_m_theta), axis=-1)
        projections.append(projected_vectors)
    return torch.stack(projections, axis=0)

def evaluate_functions_on_theta(theta, coefficients_list, m_values):
    evaluated_function = torch.zeros(theta.shape, dtype=torch.float32)
    for (a_cos, a_sin), m in zip(coefficients_list, m_values):
        evaluated_function += a_sin * torch.sin(m * theta) + a_cos * torch.cos(m * theta)
    return evaluated_function

In [ ]:
from scipy.ndimage import gaussian_filter
def mask_and_blur_images(array):
    '''masks signal inside radius of 14 and outside radius of 30 and adds gaussian blur for all intensity data'''
    for i in range(0,60):
        x,y = np.meshgrid(np.arange(128), np.arange(128))
        radius = np.sqrt((x-64)**2 + (y-62)**2)
        mask1 = radius <= 14
        mask2 = radius >= 30
        masked_data = array[i].copy()
        masked_data[mask1] = 0
        masked_data2 = masked_data.copy()
        masked_data2[mask2] = 0
        # masked_data_norm = (masked_data - np.min(masked_data) / (np.max(masked_data) - np.min(masked_data)))
        blurred_data = gaussian_filter(masked_data2, sigma=.65)
        array[i] = blurred_data
    return array

### Note, I think the columns and rows are showing x vs y respectively (opposite how we normally plot)
#### We can double check this later

## Load data and optimize offset

In [ ]:
# Define parameters
ms = torch.arange(12)
angles = torch.arange(0, 6) * 2 * torch.pi / 6.

# Extract data_theta
data = np.load('/Users/cadenmyers/billingelab/dev/skyrmion_lattices/images/image_111019.npz')['data']
data_theta = torch.atan2(torch.tensor(data[1]), torch.tensor(data[0]))

# Extract data file paths
file_path = '/Users/cadenmyers/billingelab/dev/skyrmion_lattices/tests/'
files = ['118923.npz', '119486.npz', '119996.npz', '120506.npz', '121016.npz', '121405.npz', '121855.npz', '122365.npz', '122875.npz']

# Extract Training data (can ignore)
# Extract data intensity and phi (unmasked)
# training_data = np.load('/Users/yucongchen/billingegroup/skyrmion-lattices-data/training_data.npz')
# data_intensity = torch.tensor(training_data['intensity'])
# data_intensity = torch.tensor(mask_and_blur_images(training_data['intensity']))
# Preprocess phi to get angle difference
# phi = training_data['phi'] - 253.1473
# print("phi:", phi)

In [ ]:
# Made offset a required argument
def optimize_offset(intensity, offset):
    max_iter = 101    
    opt = torch.optim.Adam([offset], lr=1e-2)
    for i in range(max_iter):
        projection = project_theta(angles + offset, ms).sum(1)
        evaluate_image_theta = evaluate_functions_on_theta(data_theta, projection, ms)
        loss = -(intensity * evaluate_image_theta).sum()
        opt.zero_grad()
        loss.backward()
        opt.step()
        #if i % 100 == 0:
        #    print(loss.item(), offset.item())
    return offset, evaluate_image_theta

In [ ]:
# Training/Testing
for file in files:
    # Read in data intensity
    data = np.load(file_path + file)
    data_intensity = torch.tensor(mask_and_blur_images(data['intensity']))
    data_numor = data['numor']
    print(data_numor)
    
    # Iterate to find offset
    offset_list, evaluate_image_theta_list = [], []
    offset = torch.tensor(0.)
    offset.requires_grad = True
    for intensity in data_intensity:
        offset, evaluate_image_theta = optimize_offset(intensity, offset)
        offset_list.append(offset.item()), evaluate_image_theta_list.append(evaluate_image_theta)
    print("offset in radius:", offset_list)

    # Plot
    fig, ax = plt.subplots(nrows=6, ncols=10, figsize=(30, 18))
    for i in range(6):
        for j in range(10):
            ax[i, j].imshow((evaluate_image_theta_list[i*6 + j] / evaluate_image_theta_list[i*6 + j].abs().max() + data_intensity[i*6 + j] / data_intensity[i*6 + j].abs().max()).detach(), cmap='plasma')
    fig.suptitle(file)
    plt.show()

    # Results
    offset_diff_degrees = []
    for offset in offset_list:
        # We're plotting y vs x instead of x vs y hence we need to use 90-offset instead of offset
        new_offset = -offset/torch.pi*180
        offset_diff_degrees.append(new_offset)
    print("offset (preprocessed) in degrees:", offset_diff_degrees)
    
    # Save offset and evaluate image theta
    np.savez('offset_' + file, numor=data['numor'], offset=offset_list)



In [ ]:
# offset
for file in files:
    # Read in data intensity
    data = np.load(file_path + 'offset_' + file)
    data_numor = data['numor']
    data_offset = data['offset']
    
    # Preprocess offset
    # Results
    offset0 = -data_offset[0]/torch.pi*180
    offset_diff_degrees = []
    for offset in data_offset:
        # We're plotting y vs x instead of x vs y hence we need to use 90-offset instead of offset
        new_offset = -offset/torch.pi*180 - offset0
        offset_diff_degrees.append(new_offset)
    print("Angles in degrees (starting numor =", data_numor[0], "):", offset_diff_degrees)
    time = np.arange(len(offset_diff_degrees)) * 10
    # np.savez(f"{file.removesuffix(".npz")}_offset.npz", time=time, offset_angle=offset_diff_degrees)

    # Plot angles
    plt.plot(time, offset_diff_degrees)
    plt.ylabel("Offset Angle")
    plt.xlabel("Time (s)")
    plt.grid(True)
    plt.title(data['numor'][0])
    plt.tick_params(direction='in')
    # plt.ylim(-30,30)
    plt.show()
